In [1]:
import numpy as np
import pandas as pd
import os
from collections import Counter

In [2]:
import datetime as dt

In [3]:
adresses = os.path.join('paradisepanamapapers', 'Addresses.csv')
all_edges = os.path.join('paradisepanamapapers', 'all_edges.csv')
entities = os.path.join('paradisepanamapapers', 'entities.csv')
intermediaires = os.path.join('paradisepanamapapers', 'intermediaries.csv')
officers = os.path.join('paradisepanamapapers', 'officers.csv')

## Adresse Dataframe 

In [5]:
df_addr = pd.read_csv(adresses, dtype = {'note': str})

In [6]:
print('The dataset involve {} entries'.format(df_addr.size))

The dataset involve 1212840 entries


In [7]:
df_addr.head()

,address,icij_id,valid_until,country_codes,countries,node_id,sourceID,note
0,-\t27 ROSEWOOD DRIVE #16-19 SINGAPORE 737920,6991059DFFB057DF310B9BF31CC4A0E6,The Panama Papers data is current through 2015,SGP,Singapore,14000001,Panama Papers,NaN
1,"""Almaly Village"" v.5, Almaty Kazakhstan",C65A269424AA1A8BBAC416FA30150938,The Panama Papers data is current through 2015,KAZ,Kazakhstan,14000002,Panama Papers,NaN
2,"""Cantonia"" South Road St Georges Hill Weybridg...",240EE44DFB70AF775E6CD02AF8CB889B,The Panama Papers data is current through 2015,GBR,United Kingdom,14000003,Panama Papers,NaN
3,"""CAY-OS"" NEW ROAD; ST.SAMPSON; GUERNSEY; CHANN...",340C6D3D2FC60AE58C53BB3618129C43,The Panama Papers data is current through 2015,GGY,Guernsey,14000004,Panama Papers,NaN
4,"""Chirag"" Plot No 652; Mwamba Road; Kizingo; Mo...",6DBCD74F5AAB63EABC166885CE8CD7CD,The Panama Papers data is current through 2015,KEN,Kenya,14000005,Panama Papers,NaN


In [8]:
Counter(df_addr['note'])

Counter({nan: 151582, 'Record manually added from leaked documents': 23})

In [9]:
Counter(df_addr['valid_until'])

Counter({'The Bahamas Leaks data is current through early 2016.': 551,
         'The Offshore Leaks data is current through 2010': 57600,
         'The Panama Papers  data is current through 2015': 93454})

In [10]:
Counter(df_addr['sourceID'])

Counter({'Bahamas Leaks': 551,
         'Offshore Leaks': 57600,
         'Panama Papers': 93454})

- valid_until is irrevelant since there is no more information than in sourceID so we will drop this col
- node_id is the id of a node who symbolise a company or individual
- sourceID is the leak information come from
- note indicated if the record was manually added - only 23 record have this note - so we can also drop this col

In [11]:
df_addr = df_addr.drop('valid_until', 1)

In [12]:
df_addr['sourceID'] =  df_addr['sourceID'].astype('category')
df_addr['country_codes'] =  df_addr['country_codes'].astype('category')
df_addr['countries'] =  df_addr['countries'].astype('category')

In [13]:
df_addr.dtypes

address            object
icij_id            object
country_codes    category
countries        category
node_id             int64
sourceID         category
note               object
dtype: object

## Edges Dataframe 

In [14]:
df_edges = pd.read_csv(all_edges)

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [15]:
df_edges.head()

,node_1,rel_type,node_2,sourceID,valid_until,start_date,end_date
0,11000001,intermediary of,10208879,Panama Papers,The Panama Papers data is current through 2015,NaN,NaN
1,11000001,intermediary of,10198662,Panama Papers,The Panama Papers data is current through 2015,NaN,NaN
2,11000001,intermediary of,10159927,Panama Papers,The Panama Papers data is current through 2015,NaN,NaN
3,11000001,intermediary of,10165779,Panama Papers,The Panama Papers data is current through 2015,NaN,NaN
4,11000001,intermediary of,10152967,Panama Papers,The Panama Papers data is current through 2015,NaN,NaN


In [16]:
print('The dataset involve {} entries'.format(df_edges.size))

The dataset involve 10748864 entries


- Rel_type is the type of the relation btw two entities (nodes) ex: 12000001 shareholder of 10073324
- node_1 and node_2 are entities (company or individual)
- start and end date are the date of beginning and end of the relationship

In [17]:
Counter(df_edges['sourceID'])

Counter({'Panama Papers': 674102,
         'Offshore Leaks': 561393,
         'Bahamas Leaks': 249190,
         nan: 50867})

In [18]:
Counter(df_edges['valid_until'])

Counter({'The Panama Papers data is current through 2015': 668691,
         'The Offshore Leaks data is current through 2010': 561393,
         'The Bahamas Leaks data is current through early 2016.': 249190,
         'The Panama Papers  data is current through 2015': 101,
         nan: 56177})

Again we can drop the valid_until col - note: there is a typo on The Panama Papers data is current through 2015. - And here we don't have the same number for Panama papers SourceID and The Panama Papers data is current through 2015 valid_until

In [19]:
df_edges = df_edges.drop('valid_until', 1)

In [20]:
df_edges['start_date'] = pd.to_datetime(df_edges['start_date'], errors = 'coerce')
df_edges['end_date'] = pd.to_datetime(df_edges['end_date'], errors = 'coerce')

In [21]:
df_edges['sourceID'] =  df_edges['sourceID'].astype('category')

## Entities Dataframe 

In [5]:
df_entities = pd.read_csv(entities, dtype = {'original_name': str, 'former_name':str,'company_type':str, 'address':str, 'status': str,'service_provider': str, 'country_codes':str, 'countries': str, 'note':str })

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
df_entities.dtypes

name                         object
original_name                object
former_name                  object
jurisdiction                 object
jurisdiction_description     object
company_type                 object
address                      object
internal_id                 float64
incorporation_date           object
inactivation_date            object
struck_off_date              object
dorm_date                    object
status                       object
service_provider             object
ibcRUC                       object
country_codes                object
countries                    object
note                         object
valid_until                  object
node_id                       int64
sourceID                     object
dtype: object

In [7]:
print('The dataset involve {} entries'.format(df_entities.size))

The dataset involve 10395798 entries


In [13]:
df_entities.columns

Index(['name', 'original_name', 'former_name', 'jurisdiction',
       'jurisdiction_description', 'company_type', 'address', 'internal_id',
       'incorporation_date', 'inactivation_date', 'struck_off_date',
       'dorm_date', 'status', 'service_provider', 'ibcRUC', 'country_codes',
       'countries', 'note', 'valid_until', 'node_id', 'sourceID'],
      dtype='object')

In [8]:
df_entities

,name,original_name,former_name,jurisdiction,jurisdiction_description,company_type,address,internal_id,incorporation_date,inactivation_date,...,dorm_date,status,service_provider,ibcRUC,country_codes,countries,note,valid_until,node_id,sourceID
0,"TIANSHENG INDUSTRY AND TRADING CO., LTD.","TIANSHENG INDUSTRY AND TRADING CO., LTD.",NaN,SAM,Samoa,NaN,ORION HOUSE SERVICES (HK) LIMITED ROOM 1401; 1...,1001256.0,23-MAR-2006,18-FEB-2013,...,NaN,Defaulted,Mossack Fonseca,25221,HKG,Hong Kong,NaN,The Panama Papers data is current through 2015,10000001,Panama Papers
1,"NINGBO SUNRISE ENTERPRISES UNITED CO., LTD.","NINGBO SUNRISE ENTERPRISES UNITED CO., LTD.",NaN,SAM,Samoa,NaN,ORION HOUSE SERVICES (HK) LIMITED ROOM 1401; 1...,1001263.0,27-MAR-2006,27-FEB-2014,...,NaN,Defaulted,Mossack Fonseca,25249,HKG,Hong Kong,NaN,The Panama Papers data is current through 2015,10000002,Panama Papers
2,"HOTFOCUS CO., LTD.","HOTFOCUS CO., LTD.",NaN,SAM,Samoa,NaN,ORION HOUSE SERVICES (HK) LIMITED ROOM 1401; 1...,1000896.0,10-JAN-2006,15-FEB-2012,...,NaN,Defaulted,Mossack Fonseca,24138,HKG,Hong Kong,NaN,The Panama Papers data is current through 2015,10000003,Panama Papers
3,"SKY-BLUE GIFTS & TOYS CO., LTD.","SKY-BLUE GIFTS & TOYS CO., LTD.",NaN,SAM,Samoa,NaN,ORION HOUSE SERVICES (HK) LIMITED ROOM 1401; 1...,1000914.0,06-JAN-2006,16-FEB-2009,...,NaN,Defaulted,Mossack Fonseca,24012,HKG,Hong Kong,NaN,The Panama Papers data is current through 2015,10000004,Panama Papers
4,FORTUNEMAKER INVESTMENTS CORPORATION,FORTUNEMAKER INVESTMENTS CORPORATION,NaN,SAM,Samoa,NaN,LOYAL PORT LIMITED 8/F; CRAWFORD TOWER 99 JERV...,1001266.0,19-APR-2006,15-MAY-2009,...,NaN,Changed agent,Mossack Fonseca,R25638,HKG,Hong Kong,NaN,The Panama Papers data is current through 2015,10000005,Panama Papers
5,8808 HOLDING LIMITED,8808 HOLDING LIMITED (EX-DIAMOND LIMITED),DIAMOND LIMITED,SAM,Samoa,NaN,TWC MANAGEMENT LIMITED SUITE D; 19/F RITZ PLAZ...,1000916.0,05-JAN-2006,NaN,...,NaN,Active,Mossack Fonseca,23835,HKG,Hong Kong,NaN,The Panama Papers data is current through 2015,10000006,Panama Papers
6,KENT DEVELOPMENT LIMITED,KENT DEVELOPMENT LIMITED,NaN,SAM,Samoa,NaN,ORION HOUSE SERVICES (HK) LIMITED ROOM 1401; 1...,1000022.0,26-JAN-2004,03-MAY-2006,...,NaN,Defaulted,Mossack Fonseca,15757,HKG,Hong Kong,NaN,The Panama Papers data is current through 2015,10000007,Panama Papers
7,BONUS TRADE LIMITED,BONUS TRADE LIMITED,NaN,SAM,Samoa,NaN,ORION HOUSE SERVICES (HK) LIMITED ROOM 1401; 1...,1000049.0,13-FEB-2004,16-FEB-2009,...,NaN,Defaulted,Mossack Fonseca,15910,HKG,Hong Kong,NaN,The Panama Papers data is current through 2015,10000008,Panama Papers
8,AMARANDAN LTD.,AMARANDAN LTD.,NaN,SAM,Samoa,NaN,ORION HOUSE SERVICES (HK) LIMITED ROOM 1401; 1...,1000024.0,26-JAN-2004,03-MAY-2006,...,NaN,Defaulted,Mossack Fonseca,15759,HKG,Hong Kong,NaN,The Panama Papers data is current through 2015,10000009,Panama Papers
9,NEW IDEA LIMITED,NEW IDEA LIMITED,NaN,SAM,Samoa,NaN,ORION HOUSE SERVICES (HK) LIMITED ROOM 1401; 1...,1000079.0,30-MAR-2004,27-FEB-2007,...,NaN,Defaulted,Mossack Fonseca,16462,HKG,Hong Kong,NaN,The Panama Papers data is current through 2015,10000010,Panama Papers


- name, original name, and former_name when there was a change are string
- jurisdiction: country code of the country of jurisdiction - XXX when nan values
- jurisdiction_description: name of the country - redondant from previous one
- compagny_type: string describe the compagny type (ex: Standard International Company) take care some error 
- internal_id ? 
- incorporation_date: The date when an offshore entity was created.
- inactivation_date: The date when a client told the agent to deactivate the offshore entity, which could be reactivated at a later date.
- struck_off_date: A company becomes struck off when it fails to be in good standing, which happens when it fails to pay license fees. In the offshore world this is the equivalent to closing an entity, although it can be reactivated at a later date if the fees start being paid again.
- dorm_date: The date when an offshore entity stopped being active.
- statuts: information that relates to the jurisdiction where an entity may have fiscal duties
- service_provider: who provide the leaks
- ibcRUC: 

In [14]:
df_entities['incorporation_date'] = pd.to_datetime(df_entities['incorporation_date'], errors = 'coerce')
df_entities['inactivation_date'] = pd.to_datetime(df_entities['inactivation_date'], errors = 'coerce')
df_entities['struck_off_date'] = pd.to_datetime(df_entities['struck_off_date'], errors = 'coerce')
df_entities['dorm_date'] = pd.to_datetime(df_entities['dorm_date'], errors = 'coerce')